# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacationpy_df=pd.read_csv("clean_city_data.csv")
vacationpy_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Date
0,Busselton,AU,-33.6500,115.3333,48.99,98,93,13.09,1619725765
1,Ushuaia,AR,-54.8000,-68.3000,42.80,56,20,3.44,1619725535
2,Touros,BR,-5.1989,-35.4608,80.60,83,75,9.22,1619726109
3,Hermanus,ZA,-34.4187,19.2345,57.00,72,96,1.01,1619726110
4,Nikolskoye,RU,59.7035,30.7861,33.75,52,0,2.24,1619726110
...,...,...,...,...,...,...,...,...,...
571,Ōme,JP,35.7839,139.2431,56.10,100,1,2.98,1619725926
572,Waupun,US,43.6333,-88.7296,66.47,39,75,9.22,1619725927
573,Catherinas Lust,GY,6.4000,-57.6000,77.20,83,91,5.08,1619725927
574,Utrān,IN,21.2333,72.8667,87.28,52,76,9.08,1619725927


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = vacationpy_df[["Latitude", "Longitude"]].astype(float)

humidity = vacationpy_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Conditions: A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph, Zero cloudiness.
filtered_weather_df=vacationpy_df
filtered_weather_df = filtered_weather_df.dropna()
filtered_weather_df= filtered_weather_df.loc[(filtered_weather_df["Temp"]>70) & (filtered_weather_df["Temp"]<80) & (filtered_weather_df["Wind Speed"]<10) & (filtered_weather_df["Cloudiness"]==0)]
filtered_weather_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Date
161,Beloha,MG,-25.1667,45.0500,73.74,74,0,7.38,1619725822
167,Morón,AR,-34.6534,-58.6198,74.73,41,0,9.22,1619725531
354,Chino Valley,US,34.7575,-112.4538,76.64,16,0,5.01,1619725871
438,Karratha,AU,-20.7377,116.8463,79.16,36,0,8.59,1619725892
465,Chivilcoy,AR,-34.8957,-60.0167,75.99,67,0,3.00,1619725899
469,Departamento de Maldonado,UY,-34.6667,-54.9167,70.77,56,0,9.22,1619725900
492,Ambovombe,MG,-25.1667,46.0833,73.76,76,0,6.08,1619725906
526,Kiryat Gat,IL,31.6100,34.7642,71.98,69,0,7.45,1619725744


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=filtered_weather_df
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
161,Beloha,MG,-25.1667,45.0500,73.74,74,0,7.38,1619725822,
167,Morón,AR,-34.6534,-58.6198,74.73,41,0,9.22,1619725531,
354,Chino Valley,US,34.7575,-112.4538,76.64,16,0,5.01,1619725871,
438,Karratha,AU,-20.7377,116.8463,79.16,36,0,8.59,1619725892,
465,Chivilcoy,AR,-34.8957,-60.0167,75.99,67,0,3.00,1619725899,
469,Departamento de Maldonado,UY,-34.6667,-54.9167,70.77,56,0,9.22,1619725900,
492,Ambovombe,MG,-25.1667,46.0833,73.76,76,0,6.08,1619725906,
526,Kiryat Gat,IL,31.6100,34.7642,71.98,69,0,7.45,1619725744,


In [7]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params= {"radius": 5000, "types": "hotel", "key": g_key}
# params["location"]=f"{31.6100},{34.7642}"
# response=requests.get(base_url, params=params).json()
# response

In [28]:
hotel_df.columns
hotel_df = hotel_df.rename(columns={'Wind Speed': 'Wind_Speed', 'Hotel Name': 'Hotel_Name'})

In [30]:


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params= {"radius": 5000, "types": "hotel", "key": g_key}


for row in hotel_df:
    lat=hotel_df["Latitude"]
    long=hotel_df["Longitude"]
    params['location']=f"{lat},{long}"
    response=requests.get(base_url, params=params)
    jsn=response.json()
    
    #results = response['results']

    try:
        hotel_df.loc[index, 'Hotel Name'] = jsn['results'][0]['name']
        
    except:
        print("Not Found")
        

#hotel_df["Hotel Name"] = hotel_name
#hotel_df = hotel_df.dropna(how='any')
hotel_df
    

Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found


,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind_Speed,Date,Hotel_Name
161,Beloha,MG,-25.1667,45.0500,73.74,74.0,0.0,7.38,1.619726e+09,
167,Morón,AR,-34.6534,-58.6198,74.73,41.0,0.0,9.22,1.619726e+09,
354,Chino Valley,US,34.7575,-112.4538,76.64,16.0,0.0,5.01,1.619726e+09,
438,Karratha,AU,-20.7377,116.8463,79.16,36.0,0.0,8.59,1.619726e+09,
465,Chivilcoy,AR,-34.8957,-60.0167,75.99,67.0,0.0,3.00,1.619726e+09,
469,Departamento de Maldonado,UY,-34.6667,-54.9167,70.77,56.0,0.0,9.22,1.619726e+09,
492,Ambovombe,MG,-25.1667,46.0833,73.76,76.0,0.0,6.08,1.619726e+09,
526,Kiryat Gat,IL,31.6100,34.7642,71.98,69.0,0.0,7.45,1.619726e+09,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))